# Databased used
- CREMA-D: 7,442 clips
- RAVDESS: 2,880 clips
- TESS: 2,000 clips
- SAVEE: 480 clips

This provides a total of **12,802** inputs overall, with 7 emotions.

In [ ]:
# Import libraries
# KERNEL SETUP IN VS CODE:
# conda create -n myenv python=3.12.2
# conda activate myenv

%pip install resampy tf_keras tensorflow librosa pandas matplotlib kagglehub

import kagglehub
import librosa
from librosa import feature
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

In [ ]:
# Datasets via kagglehub

cremad = kagglehub.dataset_download("ejlok1/cremad")
print("CREMA-D to dataset files:", cremad)

ravdess = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")
print("RAVDESS to dataset files:", ravdess)

tess = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")
print("RAVDESS to dataset files:", tess)

savee = kagglehub.dataset_download("ejlok1/surrey-audiovisual-expressed-emotion-savee")
print("RAVDESS to dataset files:", savee)

# Pre-processing
## CREMA-D

In [25]:
# Load CREMA-D Dataset
paths = []
emotions = []

for dirname, _, filenames in os.walk(cremad): # (dirname, subdirs, filenames)
    for filename in filenames:
        if filename.endswith('.wav'):
            paths.append(os.path.join(dirname, filename))
            emotion = filename.split('_')[2]  # Get the emotion code (e.g., 'ANG')
            emotions.append(emotion)

print(paths[:5])

print(emotions[:5])

# Create DataFrame
creamd_df = pd.DataFrame()
creamd_df['paths'] = paths
creamd_df['emotions'] = emotions

# Map emotion codes to full emotions
emotion_map = {
    'ANG': 'anger',
    'DIS': 'disgust',
    'FEA': 'fear',
    'HAP': 'happy',
    'NEU': 'neutral',
    'SAD': 'sad'
}

creamd_df['emotions'] = creamd_df['emotions'].map(emotion_map)

print(creamd_df.head())

print(creamd_df['emotions'].value_counts())

['C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_ANG_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_DIS_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_FEA_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_HAP_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_NEU_XX.wav']
['ANG', 'DIS', 'FEA', 'HAP', 'NEU']
                                               paths emotions
0  C:\Users\eskillman\.cache\kagglehub\datasets\e...    anger
1  C:\Users\eskillman\.cache\kagglehub\datasets\e...  disgust
2  C:\Users\eskillman\.cache\kagglehub\datasets\e...     fear
3  C:\Users\eskillman\.cache\kagglehub\datasets\e...    happy
4  C:\Users\eskillman\.cache\kagglehub\datasets\e...  neutral
emotions
anger      1271
d

## RAVEDESS

In [33]:
paths = []
emotions = []

for dirname, _, filenames in os.walk(ravdess): # (dirname, subdirs, filenames)
    for filename in filenames:
        if filename.endswith('.wav'):
            paths.append(os.path.join(dirname, filename))
            part = filename.split('.')[0].split('-')  # Get the emotion number (e.g., '03' = happy)
            emotions.append(int(part[2]))


print(paths[:5])
print(emotions[:5])

# Create DataFrame
ravdess_df = pd.DataFrame()
ravdess_df['paths'] = paths
ravdess_df['emotions'] = emotions


# Map emotion codes to full emotions
emotion_map = {
    1 : 'neutral',
    2 : 'calm',
    3 : 'happy',
    4 : 'sad',
    5 : 'anger',
    6 : 'fear',
    7 : 'disgust',
    8 : 'suprised'

}

ravdess_df['emotions'] = ravdess_df['emotions'].map(emotion_map)

print(ravdess_df.head())

print(ravdess_df['emotions'].value_counts())

['C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Actor_01\\03-01-01-01-01-01-01.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Actor_01\\03-01-01-01-01-02-01.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Actor_01\\03-01-01-01-02-01-01.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Actor_01\\03-01-01-01-02-02-01.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Actor_01\\03-01-02-01-01-01-01.wav']
[1, 1, 1, 1, 2]
                                               paths emotions
0  C:\Users\eskillman\.cache\kagglehub\datasets\u...  neutral
1  C:\Users\eskillman\.cache\kagglehub\datasets\u...  neutral
2  C:\Users\eskillman\.cache\kagglehub\datasets\u...  neutral


## TESS

In [42]:
paths = []
emotions = []

for dirname, _, filenames in os.walk(tess): # (dirname, subdirs, filenames)
    for filename in filenames:
        if filename.endswith('.wav'):
            paths.append(os.path.join(dirname, filename))
            emotion = filename.split('.')[0].split('_')[2]  # Get the emotion code (e.g., 'ANG')
            emotions.append(emotion)

print(paths[:5])

print(emotions[:5])

# Create DataFrame
tess_df = pd.DataFrame()
tess_df['paths'] = paths
tess_df['emotions'] = emotions

# Map emotion codes to full emotions
emotion_map = {
    'angry': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'happy': 'happy',
    'neutral': 'neutral',
    'ps' : 'suprised',
    'sad' : 'sad'
}

tess_df['emotions'] = tess_df['emotions'].map(emotion_map)

print(tess_df.head())

print(tess_df['emotions'].value_counts())

['C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\toronto-emotional-speech-set-tess\\versions\\1\\TESS Toronto emotional speech set data\\OAF_angry\\OAF_back_angry.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\toronto-emotional-speech-set-tess\\versions\\1\\TESS Toronto emotional speech set data\\OAF_angry\\OAF_bar_angry.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\toronto-emotional-speech-set-tess\\versions\\1\\TESS Toronto emotional speech set data\\OAF_angry\\OAF_base_angry.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\toronto-emotional-speech-set-tess\\versions\\1\\TESS Toronto emotional speech set data\\OAF_angry\\OAF_bath_angry.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\toronto-emotional-speech-set-tess\\versions\\1\\TESS Toronto emotional speech set data\\OAF_angry\\OAF_bean_angry.wav']
['angry', 'angry', 'angry', 'angry', 'angry']
                                               paths e

## SAVEE

paths = []
emotions = []

for dirname, _, filenames in os.walk(savee): # (dirname, subdirs, filenames)
    for filename in filenames:
        if filename.endswith('.wav'):
            paths.append(os.path.join(dirname, filename))
            emotion = filename  # Get the emotion code (e.g., 'ANG')
            emotions.append(emotion)

print(paths[:5])

print(emotions[:5])

# Create DataFrame
savee_df = pd.DataFrame()
savee_df['paths'] = paths
savee_df['emotions'] = emotions

# Map emotion codes to full emotions
emotion_map = {
    'angry': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'happy': 'happy',
    'neutral': 'neutral',
    'ps' : 'suprised',
    'sad' : 'sad'
}

savee_df['emotions'] = savee_df['emotions'].map(emotion_map)

print(savee_df.head())

print(savee_df['emotions'].value_counts())